In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [2]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [8]:
image=Image.open('data/train1/1/Training_10598340.jpg')
image.size

(48, 48)

In [3]:
import os
train_generator = train_data_gen.flow_from_directory(
        "data/train1",
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [4]:
validation_generator = validation_data_gen.flow_from_directory(
        "data/test1",
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [5]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [6]:
cv2.ocl.setUseOpenCL(False)

In [7]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate =0.0001, decay=1e-6), metrics=['accuracy'])

In [8]:
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709//64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178//64)

Epoch 1/50
448/448 [==============================] - 522s 1s/step - loss: 1.7998 - accuracy: 0.2593 - val_loss: 1.6920 - val_accuracy: 0.3472
Epoch 2/50
448/448 [==============================] - 205s 457ms/step - loss: 1.6186 - accuracy: 0.3696 - val_loss: 1.5247 - val_accuracy: 0.4220
Epoch 3/50
448/448 [==============================] - 208s 463ms/step - loss: 1.5180 - accuracy: 0.4164 - val_loss: 1.4505 - val_accuracy: 0.4417
Epoch 4/50
448/448 [==============================] - 203s 452ms/step - loss: 1.4421 - accuracy: 0.4481 - val_loss: 1.3832 - val_accuracy: 0.4741
Epoch 5/50
448/448 [==============================] - 202s 450ms/step - loss: 1.3815 - accuracy: 0.4761 - val_loss: 1.3521 - val_accuracy: 0.4817
Epoch 6/50
448/448 [==============================] - 206s 459ms/step - loss: 1.3306 - accuracy: 0.4968 - val_loss: 1.2899 - val_accuracy: 0.5102
Epoch 7/50
448/448 [==============================] - 206s 460ms/step - loss: 1.2870 - accuracy: 0.5139 - val_loss: 1.2586 - va

In [9]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)


emotion_model.save_weights('emotion_model.h5')